In [18]:
import numpy as np
import pandas as pd
import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt
from termcolor import colored

import json
import os
import math
import os.path

In [19]:
%%time

folder='data/toJson_output/'
# file='swimMovementFP_ds0_nm_80_cli10_vec.json'
path_list=os.listdir(folder)
path_list.sort()

VL=400 # total video duration
seglen=5 #segment length

clients_overSegs25=[] #clients‘ video with at least 25segments

for file in path_list:
    if file[-4:]=='json':
        with open(folder+file) as f:
            data_dict = json.load(f)
            
            playedSegs=len(data_dict['I13']['segments']) #played or downoaed segments
            
            #if th video played more than 25, record it
            if playedSegs>=25:
                clients_overSegs25.append(file[:-5])
            
            #stallingPostion is the played_time, i.e., the sum of all segments' durations, which could be downloaded and played back
            stallingPostion=playedSegs*seglen #played time
            #stallingDuration is the remaining_video_time, i.e., the sum of all segments' durations, which COULD NOT be downloaded
            stallingDuration=VL-stallingPostion #remianing video duration
            
            
            # if the video is fully played, just ignore them
            if stallingDuration>0:
                #add the atificial stalling event before adding the atificial segment
                data_dict['I23']['stalling'].append([stallingPostion,stallingDuration])
                
                # Duplicate the last segment
                data_dict['I13']['segments'].append(data_dict['I13']['segments'][-1].copy())
                # set the previous last segment's duration with 5sec.
                data_dict['I13']['segments'][-2]['duration']=seglen
                # set the start time of duplicate on with 5 seconds higher
                data_dict['I13']['segments'][-1]['start']=data_dict['I13']['segments'][-2]['start']+seglen      

                
                # wirte the dict back
                with open(folder+file, 'w+') as outfile:
                    json.dump(data_dict,outfile,sort_keys=False, indent=4, separators=(',', ':'))
            else:
                pass

            
dclients_overSegs25=pd.DataFrame(clients_overSegs25)
dclients_overSegs25.to_csv('clients_overSegs25.csv',index=False)

CPU times: user 23.4 s, sys: 5.32 s, total: 28.7 s
Wall time: 37.3 s
